## Translation Experiments

### Importing Relevant Models

In [5]:
import os
import sys

sys.path.append("../")

from dotenv import load_dotenv

load_dotenv("../envs/melange.env")

True

In [8]:
# from mega.models.completion_models import model_completion
from langchain.prompts import load_prompt
from langchain.prompts.example_selector import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import FewShotPromptTemplate

from datasets import load_dataset

c:\Users\t-iwatts\AppData\Local\anaconda3\envs\mega\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataset Loading

#### In-22 Gen

In [9]:
dataset_gen = load_dataset("ai4bharat/IN22-Gen", "eng_Latn-hin_Deva", split="gen")
dataset_gen

Generating gen split: 1024 examples [00:00, 2825.42 examples/s]


Dataset({
    features: ['id', 'context', 'source', 'url', 'domain', 'num_words', 'bucket', 'sentence_eng_Latn', 'sentence_hin_Deva'],
    num_rows: 1024
})

In [10]:
dataset_gen[0]

{'id': 1,
 'context': 'A uniform is often viewed as projecting a positive image of an organisation. Maintaining personal hygiene is also an important aspect of personal appearance and dressing. An appearance is a bunch of attributes related with the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.',
 'source': 'web',
 'url': '',
 'domain': 'culture',
 'num_words': 24,
 'bucket': '18 - 25',
 'sentence_eng_Latn': 'An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.',
 'sentence_hin_Deva': 'सेवा संबंधी लोगों के लिए भेष कई गुणों का संयोजन है, जैसे कि उनके जूते, कपड़े, टाई, आभूषण, केश शैली, मेक-अप, घड़ी, कॉस्मेटिक, इत्र, आदि।'}

In [6]:
final_dataset_gen = dict()

for i, val in enumerate(dataset_gen):
    final_dataset_gen[i] = dict()
    final_dataset_gen[i]["id"] = val["id"]
    final_dataset_gen[i]["context"] = val["context"]

    for key in val.keys():
        if "sentence" in key:
            final_dataset_gen[i][key[9:]] = val[key]
            IN22_LANGUAGES.append(key[9:])

In [7]:
IN22_LANGUAGES

['asm_Beng',
 'ben_Beng',
 'brx_Deva',
 'doi_Deva',
 'eng_Latn',
 'gom_Deva',
 'guj_Gujr',
 'hin_Deva',
 'kan_Knda',
 'kas_Arab',
 'mai_Deva',
 'mal_Mlym',
 'mar_Deva',
 'mni_Mtei',
 'npi_Deva',
 'ory_Orya',
 'pan_Guru',
 'san_Deva',
 'sat_Olck',
 'snd_Deva',
 'tam_Taml',
 'tel_Telu',
 'urd_Arab',
 'asm_Beng',
 'ben_Beng',
 'brx_Deva',
 'doi_Deva',
 'eng_Latn',
 'gom_Deva',
 'guj_Gujr',
 'hin_Deva',
 'kan_Knda',
 'kas_Arab',
 'mai_Deva',
 'mal_Mlym',
 'mar_Deva',
 'mni_Mtei',
 'npi_Deva',
 'ory_Orya',
 'pan_Guru',
 'san_Deva',
 'sat_Olck',
 'snd_Deva',
 'tam_Taml',
 'tel_Telu',
 'urd_Arab',
 'asm_Beng',
 'ben_Beng',
 'brx_Deva',
 'doi_Deva',
 'eng_Latn',
 'gom_Deva',
 'guj_Gujr',
 'hin_Deva',
 'kan_Knda',
 'kas_Arab',
 'mai_Deva',
 'mal_Mlym',
 'mar_Deva',
 'mni_Mtei',
 'npi_Deva',
 'ory_Orya',
 'pan_Guru',
 'san_Deva',
 'sat_Olck',
 'snd_Deva',
 'tam_Taml',
 'tel_Telu',
 'urd_Arab',
 'asm_Beng',
 'ben_Beng',
 'brx_Deva',
 'doi_Deva',
 'eng_Latn',
 'gom_Deva',
 'guj_Gujr',
 'hin_Deva',

In [8]:
final_dataset_gen[0]

{'id': 1,
 'context': 'A uniform is often viewed as projecting a positive image of an organisation. Maintaining personal hygiene is also an important aspect of personal appearance and dressing. An appearance is a bunch of attributes related with the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.',
 'asm_Beng': 'এটা বাহ্যিক ৰূপ হৈছে সশস্ত্র বাহিনীৰ জোৱানগৰাকীৰ সৈতে জড়িত কেইবাটাও বৈশিষ্ট, যেনে তেওঁৰ জোতা, কাপোৰ, টাই, অলংকাৰ, কেশ-বিন্যাস, মেক-আপ, ঘড়ী, প্রসাধনী, সুগন্ধি, ইত্যাদি।',
 'ben_Beng': 'বাহ্য রূপ হল সেই পরিষেবা প্রদানকারী মানুষ সম্বন্ধীয় এক গুচ্ছ বৈশিষ্ট্য যেমন, তার জুতো, জামাকাপড়, টাই, গয়না, চুলের ধরন, রূপসজ্জা, ঘড়ি, প্রসাধনী, সুগন্ধি, ইত্যাদি।',
 'brx_Deva': 'नुजानाया खामानिनि सुबुंजों सोमोन्दो गोनां महरफोरनि मोनसे जथाइ, जेरै बिसोरनि जुता, गस्ला, टाइ, गहेना, खानायनि स्टाइल, मेक-आप, घोरि, देलाइग्रा, सेन, बायदि बायदि।',
 'doi_Deva': 'इक दखौट सेवा करने आह्\u200c\u200c\u200cले व्यक्ति कन्नै सरबंधत खासियतें दा इक समूह्

#### Few Shot Dataset (FLORES-22)

In [11]:
data = load_dataset("facebook/flores", "eng_Latn", split="dev")
data

Generating dev split: 997 examples [00:00, 5702.02 examples/s]
Generating devtest split: 1012 examples [00:00, 4165.71 examples/s]


Dataset({
    features: ['id', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink', 'sentence'],
    num_rows: 997
})

In [9]:
few_shot_dataset = dict()

In [10]:
for language in IN22_LANGUAGES:
    try:
        data = load_dataset("facebook/flores", language, split="dev")
        few_shot_dataset[language] = dict()

        for val in enumerate(data):
            few_shot_dataset[language][val["id"]] = val["sentence"]
    except:
        print("Language not Found")
    break

Dataset({
    features: ['id', 'URL', 'domain', 'topic', 'has_image', 'has_hyperlink', 'sentence'],
    num_rows: 997
})


In [12]:
data[0]

{'id': 1,
 'URL': 'https://en.wikinews.org/wiki/Scientists_say_new_medical_diagnostic_chip_can_sort_cells_anywhere_with_an_inkjet',
 'domain': 'wikinews',
 'topic': 'health',
 'has_image': 0,
 'has_hyperlink': 0,
 'sentence': "সোমবাৰে, ষ্টেনফ'ৰ্ড ইউনিভাৰচিটি স্কুল অৱ মেডিচিনৰ বিজ্ঞানীসকলে  ঘোষণা কৰিছিল যে তেওঁলোকে এটা নতুন ৰোগ চিনাক্তকৰণ সঁজুলি আৱিষ্কাৰ কৰিছিল যিয়ে ধৰণ অনুসৰি কোষসমূহ ক্ৰম কৰে: এটা ক্ষুদ্ৰ মুদ্ৰণযোগ্য চিপ যিটো প্ৰত্যেকটোতে সম্ভৱপৰ প্ৰায় 1 U.S. চেণ্ট খৰচ হোৱাকৈ ষ্টেণ্ডাৰ্ড ইনজেক্ট প্ৰিণ্টাৰ ব্যৱহাৰ কৰি প্ৰস্তুত কৰা হৈছে।"}

In [42]:
few_shot_dataset = load_dataset("facebook/flores")

ValueError: Config name is missing.
Please pick one among the available configs: ['ace_Arab', 'bam_Latn', 'dzo_Tibt', 'hin_Deva', 'khm_Khmr', 'mag_Deva', 'pap_Latn', 'sot_Latn', 'tur_Latn', 'ace_Latn', 'ban_Latn', 'ell_Grek', 'hne_Deva', 'kik_Latn', 'mai_Deva', 'pbt_Arab', 'spa_Latn', 'twi_Latn', 'acm_Arab', 'bel_Cyrl', 'eng_Latn', 'hrv_Latn', 'kin_Latn', 'mal_Mlym', 'pes_Arab', 'srd_Latn', 'tzm_Tfng', 'acq_Arab', 'bem_Latn', 'epo_Latn', 'hun_Latn', 'kir_Cyrl', 'mar_Deva', 'plt_Latn', 'srp_Cyrl', 'uig_Arab', 'aeb_Arab', 'ben_Beng', 'est_Latn', 'hye_Armn', 'kmb_Latn', 'min_Arab', 'pol_Latn', 'ssw_Latn', 'ukr_Cyrl', 'afr_Latn', 'bho_Deva', 'eus_Latn', 'ibo_Latn', 'kmr_Latn', 'min_Latn', 'por_Latn', 'sun_Latn', 'umb_Latn', 'ajp_Arab', 'bjn_Arab', 'ewe_Latn', 'ilo_Latn', 'knc_Arab', 'mkd_Cyrl', 'prs_Arab', 'swe_Latn', 'urd_Arab', 'aka_Latn', 'bjn_Latn', 'fao_Latn', 'ind_Latn', 'knc_Latn', 'mlt_Latn', 'quy_Latn', 'swh_Latn', 'uzn_Latn', 'als_Latn', 'bod_Tibt', 'fij_Latn', 'isl_Latn', 'kon_L

In [7]:
few_shot_dataset = dict()

In [8]:
directory_path = "../translation_dataset/flores-22_dev/all"

for folder in os.listdir(directory_path):
    few_shot_dataset[folder[9:]] = list()
    for file in os.listdir(os.path.join(directory_path, folder)):
        if file[-4:] != ".csv":
            if file == "dev.eng_Latn":
                with open(
                    os.path.join(directory_path, folder, file),
                    mode="r",
                    encoding="utf-8",
                ) as file_eng:
                    data_eng = [val.strip() for val in file_eng.readlines()]
            else:
                with open(
                    os.path.join(directory_path, folder, file),
                    mode="r",
                    encoding="utf-8",
                ) as file_source:
                    data_source = [val.strip() for val in file_source.readlines()]

    for source, english in zip(data_source, data_eng):
        few_shot_dataset[folder[9:]].append({"sentence": source, "response": english})

In [9]:
few_shot_dataset["asm_Beng"]

[{'sentence': "সোমবাৰে, ষ্টেনফ'ৰ্ড ইউনিভাৰচিটি স্কুল অৱ মেডিচিনৰ বিজ্ঞানীসকলে  ঘোষণা কৰিছিল যে তেওঁলোকে এটা নতুন ৰোগ চিনাক্তকৰণ সঁজুলি আৱিষ্কাৰ কৰিছিল যিয়ে ধৰণ অনুসৰি কোষসমূহ ক্ৰম কৰে: এটা ক্ষুদ্ৰ মুদ্ৰণযোগ্য চিপ যিটো প্ৰত্যেকটোতে সম্ভৱপৰ প্ৰায় 1 U.S. চেণ্ট খৰচ হোৱাকৈ ষ্টেণ্ডাৰ্ড ইনজেক্ট প্ৰিণ্টাৰ ব্যৱহাৰ কৰি প্ৰস্তুত কৰা হৈছে।",
  'response': 'On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.'},
 {'sentence': 'শীৰ্ষ গৱেষকে কয় যে এইটোৱে কেন্সাৰ, যক্ষ্মা, এইচ.আই.ভি আৰু মেলেৰিয়াৰ প্ৰাথমিক পৰ্য্যায়ৰ চিনাক্তকৰণ নিম্ন আয়ৰ দেশবিলাকৰ ৰোগীসকললৈ আনিব পাৰে , য’ত স্তন কেন্সাৰৰ দৰে বেমাৰবোৰৰ সুস্থ হোৱাৰ হাৰ ধনী দেশৰ তুলনাত অৰ্ধেক হ’ব পাৰে।',
  'response': 'Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries,

### Prompting Templates

In [16]:
class PromptTemplate:

    def __init__(self, source, target, sentence) -> None:
        self.source = source
        self.target = target
        self.sentence = sentence

    def get_example_selector(self, criteria, few_shots, top_k):

        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")

        if criteria == "semantic":
            return SemanticSimilarityExampleSelector.from_examples(
                examples=few_shots,
                embeddings=embeddings,
                vectorstore_cls=FAISS,
                k=top_k,
            )

        elif criteria == "mmr":
            return MaxMarginalRelevanceExampleSelector.from_examples(
                examples=few_shots,
                embeddings=embeddings,
                vectorstore_cls=FAISS,
                k=top_k,
            )

    def get_query_prompt(self):
        query_prompt = load_prompt("../mega/prompting/translation_template/query.yaml")
        query_prompt = query_prompt.format(
            source=self.source, target=self.target, sentence=self.sentence
        )
        return query_prompt

    def get_few_shot_prompt(self, few_shots, criteria, top_k=8):
        example_prompt = load_prompt(
            "../mega/prompting/translation_template/few_shot.yaml"
        )

        prefix = """INSTRUCTION: Translate the following Sentence from {source} to {target} in the example format given below."""
        suffix = "Sentence: {sentence}\nResponse:"

        few_shot_prompt = FewShotPromptTemplate(
            example_selector=self.get_example_selector(criteria, few_shots, top_k),
            examples=None,
            example_prompt=example_prompt,
            prefix=prefix,
            suffix=suffix,
            input_variables=["source", "target", "sentence"],
            example_separator="\n",
        )

        return few_shot_prompt.format(
            source=self.source, target=self.target, sentence=self.sentence
        )

In [17]:
source = "Hindi"
target = "English"
sentence = dataset["hin_Deva"][0]
sentence

'सेवा संबंधी लोगों के लिए भेष कई गुणों का संयोजन है, जैसे कि उनके जूते, कपड़े, टाई, आभूषण, केश शैली, मेक-अप, घड़ी, कॉस्मेटिक, इत्र, आदि।'

In [18]:
few_shots = few_shot_dataset["hin_Deva"]
len(few_shots)

997

In [19]:
prompts = PromptTemplate(source, target, sentence)

In [20]:
prompts.get_query_prompt()

'Instruction : Translate the following Sentence from Hindi to English.\nSentence: सेवा संबंधी लोगों के लिए भेष कई गुणों का संयोजन है, जैसे कि उनके जूते, कपड़े, टाई, आभूषण, केश शैली, मेक-अप, घड़ी, कॉस्मेटिक, इत्र, आदि।\nResponse:'

In [21]:
prompt = prompts.get_few_shot_prompt(few_shots[:8], "mmr", 2)
prompt

"INSTRUCTION: Translate the following Sentence from Hindi to English in the example format given below.\nSentence: व्हाइटहॉल पर लगभग 11:00 बजे स्थानीय समय (यूटीसी + 1) पर विरोध शुरू हुआ, जो प्रधानमंत्री के आधिकारिक आवास डाउनिंग स्ट्रीट के पुलिस-पहरेदार के दरवाजे के सामने था।\nResponse: The protest started around 11:00 local time (UTC+1) on Whitehall opposite the police-guarded entrance to Downing Street, the Prime Minister's official residence.\nSentence: शोधकर्ताओं ने कहा है कि यह अल्प आय वाले देशों में कैंसर, टीवी, एचआईवी और मलेरिया के रोगियों की आसानी से पहचान करेगा, जहाँ अमीर देशों की तुलना में स्तन कैंसर जैसी बीमारी में जीवित रहने की दर आधी हो सकती है.\nResponse: Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.\nSentence: सेवा संबंधी लोगों के लिए भेष कई गुणों का संयोजन है, जैसे कि उनके जूते, कपड़े, टाई,

In [22]:
response = gpt3x_completion(prompt, "gpt-35-turbo")
response

'For service people, appearance factors include their shoes, clothes, ties, jewelry, hair styles, makeup'